# Gallery Example: Hyper/Hyper/1 Linear Network

This example demonstrates a linear network with n queues in series:
- **Arrivals**: Hyperexponential distribution
- **Service**: Hyperexponential with varying parameters per queue
- **Servers**: 1 server per queue
- **Scheduling**: FCFS
- **Topology**: Linear (tandem) network

The service times vary across queues to create a 'pyramid' utilization pattern.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_hyphyp1_linear(n=2, Umax=0.9): Network {
    """Create Hyp/Hyp/1-Linear queueing network"""
    val model = Network("Hyp/Hyp/1-Linear")
// Block 1: nodes
    val nodes = []
    val source = Source(model, "mySource")
    nodes.append(source)
    
    for i in range(n):
        val queue = Queue(model, f"Queue{i+1}", SchedStrategy.FCFS)
        nodes.append(queue)
    
    val sink = Sink(model, "mySink")
    nodes.append(sink)
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
// HyperExp arrivals
    source.setArrival(oclass, HyperExp.fit_mean_and_scv(1, 2))
// Create pyramid utilization pattern
    val half = n // 2
    val means = list(np.linspace(0.1, Umax, half))
    if n % 2 == 0:
        val means = means + means[::-1]
    else:
        val means = means + [Umax] + means[::-1]
// Set service for each queue with increasing SCV
    for i in range(n):
        nodes[i+1].setService(oclass, HyperExp.fit_mean_and_scv(means[i], 1 + i + 1))
// Block 3: topology
    model.link(Network.serialRouting(*nodes))
    
    return model
// Create the model with 2 queues
val model = gallery_hyphyp1_linear(n=2)

## About Linear (Tandem) Networks

In a linear network:
- Jobs pass through all queues in sequence
- Each queue has different service characteristics
- Bottleneck determines overall throughput

This example uses:
- 'Pyramid' mean service times: increasing then decreasing
- Increasing SCV: more variable service at later queues
- High variability in both arrivals and service (HyperExp)

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)